In [ ]:
#Data Science Project
#Authors: Raphael Ravinet, Jeffrey Giantonio, Kai Penfold

# Importing modules

In [ ]:
import os
from sklearn.tree import export_graphviz
import numpy as np
import pandas as pd
from datetime import datetime
from full_fred.fred import Fred
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
import xgboost as xgb
from sklearn.metrics import (accuracy_score, brier_score_loss, classification_report, 
confusion_matrix, mean_squared_error, precision_score, roc_auc_score,make_scorer,log_loss, f1_score, roc_curve, auc)
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import interpolate
from sklearn.tree import DecisionTreeClassifier
import optuna
from functools import partial
from xgboost import XGBClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import metrics
from IPython.display import Image
from sklearn.tree import plot_tree

# Loading the data

In [ ]:
#Connecting with Fred API key
os.environ["FRED_API_KEY"] = "40537c7208bc03361ef9a48a5f60b7e0"

fred = Fred()

# Checking if the API key is found in the environment
api_key_found = fred.env_api_key_found()
print(api_key_found) 

In [ ]:
#Importing datasets

#Fred Database
cpi_df = fred.get_series_df('CPIAUCSL',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')
ffrate_df = fred.get_series_df('FEDFUNDS',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')
gdp_df = fred.get_series_df('GDPC1',observation_start="1993-01-01", observation_end="2023-07-01", frequency = 'q')
unrate_df = fred.get_series_df('UNRATE',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')
indpro_df = fred.get_series_df('INDPRO',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')
conssen_df = fred.get_series_df('UMCSENT',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')
trade_df = fred.get_series_df('BOPGSTB',observation_start="1993-02-01", observation_end="2023-07-01", frequency = 'm')

#Yahoo Finance Database
spx = yf.Ticker('SPY')
vix = yf.Ticker('^VIX')
usdol= yf.Ticker('DX-Y.NYB')

spx_df = spx.history(period="1d", start="1993-02-01", end='2023-08-01', interval="1mo")
vix_df = vix.history(period="1d", start="1993-02-01", end='2023-08-01', interval="1mo")
usdol_df = usdol.history(period="1d", start="1993-02-01", end='2023-08-01', interval="1mo")

#MSCI Database
acwi = pd.read_excel('/Users/raphaelravinet/Downloads/ACWI index.xls')
#From Investing
comm_index = pd.read_csv('/Users/raphaelravinet/Downloads/Bloomberg Commodity Historical Data.csv')


In [ ]:
#Open the datasets we have downloaded from Guru Focus. The are all in the same folder and they are all excel files
#So, we will bulk import them

directory = '/Users/raphaelravinet/Downloads/Code/DS'
financial_data2 = {}

for file_name in os.listdir(directory):
    if file_name.endswith('.xlsx'):  
        file_path = os.path.join(directory, file_name)
        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            financial_data2[file_name] = df
        except Exception as e:
            print(f"Failed to read {file_name}: {e}")


# Preprocessing data

In [ ]:
#Creating dictionaries with the data

fred_data = {'cpi' : cpi_df, 'int_rate': ffrate_df, 'gdp' : gdp_df, 
             'unemployement': unrate_df, 'indpro': indpro_df, 
             'consumer_sentiment': conssen_df, 'trade': trade_df}
financial_data = {'spx': spx_df,'vix': vix_df, 'dollar_index': usdol_df,}

In [ ]:
#As these datasets have the same structure we will remove the text from the beginning of our file, filter for the relevant dates, and set them as index
#We are also resampling our data (as they were daily) and using the mean to do that


for file, df in financial_data2.items():
    df = df.iloc[:, :-1]
    df = df.iloc[4:]
    df.columns = ['Date', file.replace('.xlsx', '').replace('_', ' ')]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df[(df.index > pd.Timestamp('1993-01-01')) & (df.index < pd.Timestamp('2023-07-01'))]
    df = df.sort_index(ascending=True)
    df = df.resample('M').mean()
    df.index = df.index.shift(1, freq='MS')
    financial_data2[file] = df

In [ ]:
#Renaming and converting date column to date format

for name, df in fred_data.items():
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.rename(columns={'value': name}, inplace=True)

In [ ]:
#Doing the same for the financial data

for name, df in financial_data.items():
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize(None)
    df.index = df.index.normalize()
    df.rename(columns={'Close': name}, inplace=True)
    financial_data[name] = df.shift(1)


In [ ]:
#Combining all into one dataframe. Here all the data will go through based on the index
#If just one df has a value for that particular row, all others would be NA.

all_dataframes = list(fred_data.values()) + list(financial_data.values()) + list(financial_data2.values())
combined_df = pd.concat(all_dataframes, axis=1)

In [ ]:
#Checking the name of our columns
combined_df.info()

In [ ]:
#Dropping columns we do not need

combined_df = combined_df.drop(columns = ['Open', 'High', 'Low', 'Stock Splits', 
                            'Capital Gains','realtime_start', 'realtime_end'])

In [ ]:
#VIX, us dollar index and SPY dataframes, all have columns with the same name ('Volume' and 'Dividend'). So if we just
# try to drop using "standard" method, we will end up dropping all these columns, however 
#we don't want to drop the volume and dividends column from SPY as we will be using it in our feature engineering later on

combined_df = combined_df.loc[:,~combined_df.columns.duplicated()].copy()
combined_df

In [ ]:
nan_val = combined_df.isnull().sum()
print(f"Number of NaN Values per Column:\n{nan_val}' ")
combined_df.info()

In [ ]:
#Converting gdp to numeric values, and taking the percentage between the quarterly points we have available 

combined_df['gdp'] = pd.to_numeric(combined_df['gdp'], errors='coerce')
combined_df['gdp'] = combined_df['gdp'].pct_change(periods=3)

In [ ]:
#Forward Filling gdp

combined_df['gdp'] = combined_df['gdp'].fillna(method='ffill')
combined_df['gdp']

In [ ]:
#Converting everything to float so we can do some data analysis and perform some operations
combined_df = combined_df.astype(float)

In [ ]:
#Renaming our columns

combined_df.rename(columns ={ 'Volume': 'spx_vol', 'S&P 500 Earnings Yield with Forward Estimate ': 'earnings_yield_FE',
       'Shiller PE Ratio for the S&P 500': 'Shiller_PE_ratio',
       'S&P 500 PE Ratio with Forward Estimate': 'PE_ratio_FE',
       'S&P 500 Dividend Yield:' : 'Dividend_yied',
       'BofA Merrill Lynch U.S. High Yield Total Return Index': 'High_yield_return'}, inplace= True)

# Adding the other 2 dataframes

In [ ]:
#First let's add clean and ad the commodity index and global markets index to our dataframe.
#As they have a very different structure, it is easier to deal with them separately and then add it to the combined df

acwi.head(10)

In [ ]:
#Slicing acwi as the first rows are text
acwi = acwi.iloc[6:-19].copy()
acwi.head()

In [ ]:
#Renaming columns, converting date column to datetime and checking for null values
acwi.columns = ['Date','acwi']
acwi['Date'] = pd.to_datetime(acwi['Date'])
acwi['acwi'] = acwi['acwi'].astype(float)
acwi.set_index('Date',inplace = True)
start_date = '1993-01-01'
end_date = '2023-05-01'
acwi_clean = acwi[(acwi.index >= start_date) & (acwi.index <= end_date)].copy()
print(f"NaN values: {acwi_clean['acwi'].isnull().sum()}")
acwi_clean

In [ ]:
#ACWI value is indexed with last date of the month
#so we'll shift one day to make it the first day of the following month as the other df start with the 1st day of the month 

acwi_clean.index = acwi_clean.index + pd.offsets.MonthBegin(1)
acwi_clean.head()

In [ ]:
#Now let's deal with the commodities df
comm_index.head(10)

In [ ]:
#Let's set the date as our index and extract Price column. Maybe we could've used %change here.

comm_index.set_index('Date', inplace = True)

comm_df = comm_index[['Price']]
comm_df = comm_df.rename(columns={'Price': 'comm_index'})
comm_df.head()

In [ ]:
# Commodities dataframe has its values inversed so we'll reverse the DataFrame order. The first rows are values
#from 2023 and the last rows are old value.
comm_df = comm_df.iloc[::-1].copy()
comm_df.tail()

In [ ]:
#Converting index to datetime format

comm_df.index = pd.to_datetime(comm_df.index)

In [ ]:
#date here is in different format, so let's convert to be in the same format

def swap_day_month(date):
    if date.day > 12:
        return pd.Timestamp(year=date.year, month=date.day, day=date.month)
    else:
        return date

comm_df.index = comm_df.index.map(swap_day_month)


In [ ]:
comm_df = comm_df.shift(1)

In [ ]:
#Aligning the indexes
comm_df.loc[combined_df.index]

In [ ]:
# Joining the dataframes on their indices
combined_df = combined_df.join(comm_df, how='left')
combined_df = combined_df.join(acwi_clean, how='left')

# Data Visualization

In [ ]:
# combined_df.hist(figsize=(16,10))
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.4, hspace=0.6)


In [ ]:
sns.set_theme(style="darkgrid")

plt.figure(figsize=(10,5))
plt.plot(combined_df.loc[combined_df.index > '1995-01-01']['spx'], marker='', color='tab:blue', linewidth=2.5, alpha=0.9, label='SPY Index')

plt.title("SPY Monthly Returns Over Time", loc='center', fontsize=14, fontweight=0, color='black')
plt.xlabel("Year")
plt.ylabel("SPY Index")

plt.legend()
plt.show()

In [ ]:
# Plotting VIX and SPY, to visualize periods of high volatility

sns.set_theme(style="darkgrid")


fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Year')
ax1.set_ylabel('SPY Index', color=color)
ax1.plot(combined_df.index, combined_df['spx'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(['SPY Index'], loc='upper left')

ax2 = ax1.twinx()  
color = 'tab:orange'
ax2.set_ylabel('VIX Level', color=color)  
ax2.plot(combined_df.index, combined_df['vix'], color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(['VIX'], loc='upper right')


plt.title("SPY and VIX Monthly Levels Over Time", loc='center', fontsize=14, fontweight=0, color='black')

fig.tight_layout()  
plt.show()


In [ ]:
ohlc_df = combined_df['spx'].resample('Y').agg(['first','max', 'min', 'last'])

volume_df = combined_df['spx_vol'].resample('Y').sum()

colors = ['green' if close >= open_ else 'red' for open_, close in zip(ohlc_df['first'], ohlc_df['last'])]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.03,
                    subplot_titles=('SPY Yearly Candlestick Chart', 'Yearly Volume'),
                    row_width=[0.2, 0.7])

fig.add_trace(go.Candlestick(x=ohlc_df.index,
                             open=ohlc_df['first'],
                             high=ohlc_df['max'],
                             low=ohlc_df['min'],
                             close=ohlc_df['last'],
                             increasing_line_color='green',
                             decreasing_line_color='red'),
              row=1, col=1)

fig.add_trace(go.Bar(x=volume_df.index, y=volume_df, marker_color=colors), row=2, col=1)

fig.update_layout(title='SPY Yearly Candlestick Chart with Volume',
                  yaxis_title='SPY Price',
                  xaxis_rangeslider_visible=False,
                  showlegend=False,
                  plot_bgcolor='white')

fig.update_xaxes(title_text="<b>Year</b>", row=2, col=1)
fig.update_yaxes(title_text="<b>Volume</b>", row=2, col=1)

fig.update_layout(margin=dict(t=100, b=100))

fig.layout.annotations[0].text = 'SPY Yearly Candlestick Chart'
fig.layout.annotations[1].text = 'Yearly Volume'

fig.show()


In [ ]:
#Plotting SPY over time, using boxplots

plt.figure(figsize = (20,13))
plt.xlabel('Year')
plt.ylabel('SPY')
sns.boxplot(data=combined_df, x=combined_df.index.year, y=combined_df['spx'])




# Feature Engineering

In [ ]:
#Making our data available one month ahead, so we do not incur in look ahead bias

projected_variables = ['cpi', 'int_rate', 'gdp', 'unemployement', 'indpro',
       'consumer_sentiment','trade']

for variable in projected_variables:
        lagged_column_name = f'{variable}_report'
        combined_df[lagged_column_name] = combined_df[variable].shift(2)



In [ ]:
combined_df

In [ ]:
#Creating annual cpi variable with last 12 months change
combined_df['cpi_annual'] = combined_df['cpi'].pct_change(periods=12)

In [ ]:
combined_df.columns

In [ ]:
#Creating our technical indicators

delta = combined_df['spx'].diff()

up = delta.clip(lower=0)
down = -1 * delta.clip(upper=0)

# Calculate the Exponential Moving Averages (EMA) of the gains and losses
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()

# Calculate the Relative Strength (RS)
rs = ema_up / ema_down

# Calculate the Relative Strength Index (RSI)
combined_df['RSI'] = 100 - (100 / (1 + rs))


# Short term exponential moving average (EMA)
combined_df['EMA_12'] = np.log(combined_df['spx']) - np.log(combined_df['spx']).ewm(span=12).mean()

# Long term exponential moving average (EMA)
combined_df['EMA_24'] = np.log(combined_df['spx']) - np.log(combined_df['spx']).ewm(span=24).mean()

# # MACD line
# combined_df['MACD'] = exp1 - exp2

# Signal line
# combined_df['Signal_Line'] = combined_df['MACD'].ewm(span=9, adjust=False).mean()

obv = pd.Series(index=combined_df.index, dtype='float64')
obv.iloc[0] = 0

# Calculate OBV
for i in range(1, len(combined_df)):
    if combined_df['spx'].iloc[i] > combined_df['spx'].iloc[i - 1]:
        obv.iloc[i] = obv.iloc[i - 1] + combined_df['spx_vol'].iloc[i]
    elif combined_df['spx'].iloc[i] < combined_df['spx'].iloc[i - 1]:
        obv.iloc[i] = obv.iloc[i - 1] - combined_df['spx_vol'].iloc[i]
    else:
        obv.iloc[i] = obv.iloc[i - 1]

combined_df['OBV'] = obv


# Moving Average
combined_df['MA_20'] = combined_df['spx'].rolling(window=20).mean()

# Standard Deviation
combined_df['STD_20'] = combined_df['spx'].rolling(window=20).std()



In [ ]:
combined_df.columns

In [ ]:
#Getting the log diff of SPY

combined_df['spx_diff'] = np.log(combined_df['spx']).diff()

In [ ]:
#Below are the transformations functions we will apply to our data

def percentage_change(column):
    return column.pct_change()

def simple_diff(column):
    return column.diff()

def log_diff(column):
    return np.log(column).diff()

def identity(column):
    return column

In [ ]:
#Columns that we will apply the transformations

transformations ={
    'cpi_report': percentage_change,
    'int_rate_report': simple_diff,
    'gdp_report': identity,
    'unemployement_report': simple_diff,
    'indpro_report': percentage_change,
    'consumer_sentiment_report': percentage_change,
    'trade_report': percentage_change,
    'spx_vol': log_diff,
    'vix': percentage_change,
    'dollar_index': percentage_change,
    'comm_index': percentage_change,
    'acwi': percentage_change,
    'cpi_annual': simple_diff,
    'S&P 500 Dividend Yield': log_diff,
    'EMA_12': identity,
    'EMA_24': identity,
    'earnings_yield_FE': percentage_change,
    'Shiller_PE_ratio': percentage_change,
    'PE_ratio_FE': percentage_change,
    'High_yield_return': percentage_change,
    'OBV': identity,
    'RSI': identity,
    'spx': identity,
    'spx_diff': identity,
    
}


In [ ]:
transformed_columns = {}
for column, transformation in transformations.items():
    transformed_columns[column] = transformation(combined_df[column])

# Combine the results into a new DataFrame
transformed_df = pd.DataFrame(transformed_columns)
transformed_df

In [ ]:
#Getting the name of the original features, before lagging some of them

original_features = transformed_df.columns.tolist()

In [ ]:
#Checking which columns we didn't carry it over to transformed df

set(list(combined_df.columns)) - set(list(transformed_df.columns))

In [ ]:
# #Checking how correlated the variables are amongst themselves

# plt.figure(figsize=(24,18))
# features_corr = sns.heatmap(transformed_df.corr(), cmap = 'RdYlGn', annot = True)

In [ ]:
transformed_df.columns

In [ ]:
#Taking the log difference between tomorrow and today's return

transformed_df['log_ret_spy'] = np.log(transformed_df['spx']).diff().shift(-1)

In [ ]:
#Creating our target variable. One without threshold and the other one with threshold.
threshold = 0.03
transformed_df['y_new'] = transformed_df['log_ret_spy'].apply(lambda x: '1' if x > threshold else '0').astype(int)
transformed_df['y'] = (transformed_df['log_ret_spy'] >= 0).astype(int)


#Creating our weight variable based on absolute return

transformed_df['weight'] = abs(transformed_df['log_ret_spy'])

In [ ]:
#Lagging our variables
variables_to_lag = ['cpi_report', 'int_rate_report', 'gdp_report', 'unemployement_report', 'indpro_report',
       'consumer_sentiment_report', 'spx_diff']

max_lag = 5
for variable in variables_to_lag:
    for lag in range(1,max_lag + 1):
        lagged_column_name = f'{variable}_lag{lag}'
        transformed_df[lagged_column_name] = transformed_df[variable].shift(lag)


In [ ]:
#Filtering our dataframe for the period we will be analysing 

transformed_df = transformed_df.loc[(transformed_df.index > '1995-01-01') & (transformed_df.index < '2023-05-01')].copy()

In [ ]:
#Our final dataframe
print(transformed_df.isnull().sum().sum())
transformed_df

In [ ]:
transformed_df.columns

In [ ]:
#Splitting into Train and test. Here we are leaving the last 40 observations for testing.
#Not that this is not our cross validation split, we will further split our train set here into multiple train and
#tests.
transformed_df_train = transformed_df.iloc[:-40].copy()
transformed_df_test = transformed_df.iloc[-40:].copy() 
transformed_df_train.columns

In [ ]:
transformed_df_train

In [ ]:
features_to_plot = ['cpi_report', 'int_rate_report', 'gdp_report', 'unemployement_report', 'indpro_report',
       'consumer_sentiment_report', 'trade_report', 'spx_vol', 'dollar_index','comm_index', 'y_new']
features_to_plot_df = transformed_df[features_to_plot]

In [ ]:
feature_to_plot_mapping = {
    'cpi_report': 'CPI',
    'comm_index': 'Commodity_index',
    'consumer_sentiment_report': 'Sentiment_index',
    'dollar_index': 'Dollar_index',
    'gdp_report': 'GDP',
    'indpro_report': 'Ind_production',
    'int_rate_report': 'Int_rate',
    'spx_vol': 'SPY Volume',
    'trade_report': 'Trade Balance',
    'unemployement_report': 'Unemployement_rate',}

In [ ]:
feature_to_plot_mapping

In [ ]:
X_to_plot = features_to_plot_df.drop('y_new', axis=1)
y_to_plot = features_to_plot_df['y_new']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_to_plot)

X_scaled_df = pd.DataFrame(X_scaled, columns=X_to_plot.columns, index=X_to_plot.index)
X_scaled_df['y_new'] = y_to_plot

X_scaled_df_renamed = X_scaled_df.rename(columns=feature_to_plot_mapping)


y_min = X_scaled_df.drop('y_new', axis=1).min().min() - 1
y_max = X_scaled_df.drop('y_new', axis=1).max().max() + 1

grouped_renamed = X_scaled_df_renamed.groupby('y_new')
for name, group in grouped_renamed:
    fig, ax = plt.subplots(figsize=(8, 6))  
    group.drop('y_new', axis=1).boxplot(rot=90, ax=ax)
    ax.set_ylim(y_min, y_max)  
    plt.title(f'Boxplot for class {name}')
    plt.show()

In [ ]:
#Checking how correlated the variables are amongst themselves

plt.figure(figsize=(16,10))
features_corr = sns.heatmap(X_scaled_df_renamed.corr(), cmap = 'RdYlGn', annot = True)

In [ ]:
#Extracting the features we will use as our independent variable
features =['cpi_report', 'int_rate_report', 'gdp_report', 'unemployement_report',
       'indpro_report', 'consumer_sentiment_report', 'trade_report', 'spx_vol',
       'dollar_index', 'comm_index', 'acwi', 'cpi_annual',
       'S&P 500 Dividend Yield', 'EMA_12', 'EMA_24', 'earnings_yield_FE',
       'Shiller_PE_ratio', 'PE_ratio_FE', 'OBV', 'RSI',
       'spx_diff','cpi_report_lag1', 'cpi_report_lag2',
       'cpi_report_lag3', 'cpi_report_lag4', 'cpi_report_lag5',
       'int_rate_report_lag1', 'int_rate_report_lag2', 'int_rate_report_lag3',
       'int_rate_report_lag4', 'int_rate_report_lag5', 'gdp_report_lag1',
       'gdp_report_lag2', 'gdp_report_lag3', 'gdp_report_lag4',
       'gdp_report_lag5', 'unemployement_report_lag1',
       'unemployement_report_lag2', 'unemployement_report_lag3',
       'unemployement_report_lag4', 'unemployement_report_lag5',
       'indpro_report_lag1', 'indpro_report_lag2', 'indpro_report_lag3',
       'indpro_report_lag4', 'indpro_report_lag5',
       'consumer_sentiment_report_lag1', 'consumer_sentiment_report_lag2',
       'consumer_sentiment_report_lag3', 'consumer_sentiment_report_lag4',
       'consumer_sentiment_report_lag5', 'spx_diff_lag1', 'spx_diff_lag2',
       'spx_diff_lag3', 'spx_diff_lag4', 'spx_diff_lag5',
          'High_yield_return']

In [ ]:
#Defining X variable according to our features
X = transformed_df_train[features]
X_test = transformed_df_test[features]

#Defining weight variable for train and test set
weight_train = transformed_df_train['weight']
weight_test = transformed_df_test['weight']

#Defining Y variable for train and test set
y_new = transformed_df_train['y_new']
y_new_test = transformed_df_test['y_new']
y = transformed_df_train['y']
y_test = transformed_df_test['y']

In [ ]:
#Creating graph for monthly return distribution
positive_above_3 = transformed_df[transformed_df['y_new'] == 1].copy()

positive_above_3['Year'] = positive_above_3.index.year.copy()

grouped = positive_above_3.groupby('Year').size().reset_index(name='counts').copy()

fig = go.Figure(data=[
    go.Bar(name='Positive Above 3%', x=grouped['Year'], y=grouped['counts'], marker_color='green')
])

fig.update_layout(title='Distribution of Monthly Returns Above 3% for S&P 500 Over Time',
                  xaxis_title='Year', yaxis_title='Number of Months')

fig.show()


In [ ]:
#This will display our class imbalance

classes, counts = np.unique(y_to_plot, return_counts=True)

classes = ['Below 3%', 'Above 3%']
counts_list = [counts[0], counts[1]]

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=classes, y=counts_list, palette=['#e41a1c', '#4daf4a'])

for i, count in enumerate(counts_list):
    ax.text(i, count, str(count), ha='center', va='bottom')

plt.title('Months with returns above or below 3%')
plt.show()


In [ ]:
#Checking if the previous return of S&P500 influence the current returns 

plot_acf(transformed_df['log_ret_spy'], lags=20) 
plt.show()

In [ ]:
plot_pacf(transformed_df['log_ret_spy'], lags=20)
plt.title('Partial Autocorrelation Log_ret_SPY')
plt.show()

# Cross Validation

In [ ]:
#Creating a dictionary with the models we will use. We will pass this on to the cross validation function

binary_models = {
    'LogisticRegression': LogisticRegression,
    'RandomForest': RandomForestClassifier,
    'GradientBoosting': GradientBoostingClassifier,
    'XGB': XGBClassifier
}

In [ ]:
#Defining our cross validation function. In this function we will pass the models we want to test, our X and y
#our weights and the number of splits in which we will split our data into training and test sets.
#We also have an optional argument which is the parameters that we may pass to our models.
#We will run first to get our baseline model

def time_series_validation_binary(model_classes, X, y, n_splits,weights = None,random_state = 50, model_configs=None):
    model_scores = {}
    feature_importance_dfs = {}
    y_proba_cv = {}
    roc_data = {}

    tscv = TimeSeriesSplit(n_splits=n_splits)

    for name, model_class in model_classes.items():
        config = model_configs.get(name, {}) if model_configs is not None else {}
        config['random_state'] = random_state

        # Initialize metrics
        accuracies = []
        log_losses = []
        importances = []
        all_importances = []
        aggregated_cm = np.zeros((2, 2), dtype=int)
        auc_scores = []
        y_proba_model = []
        split_indices = [] 
        roc_data[name] = {'fpr': [], 'tpr': [], 'auc': []}  

        for train_index, test_index in tscv.split(X):
            split_indices.append((train_index, test_index))
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            sample_weight_train = weights.iloc[train_index] if weights is not None else None
            sample_weight_test = weights.iloc[test_index] if weights is not None else None

            # Scaling the data
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Fitting the model - using parameters if we pass them on
            model = model_class(**config)
            model.fit(X_train_scaled, y_train, sample_weight=sample_weight_train)
            if name == 'RandomForest':  
                estimator = model.estimators_[0]
                
                export_graphviz(estimator, out_file=f'{name}_tree.dot', 
                                feature_names=X.columns,
                                rounded=True, proportion=False, 
                                precision=2, filled=True)
                os.system(f'dot -Tpng {name}_tree.dot -o {name}_tree.png')

        
            
            y_proba = model.predict_proba(X_test_scaled)
            y_proba_model.append(y_proba[:, 1])
            
            fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1], sample_weight=sample_weight_test)
            roc_auc = auc(fpr, tpr)

            # Storing feature importances if available - not all models would get that
            if hasattr(model, 'feature_importances_'):
                all_importances.append(model.feature_importances_)

            # Calculate and store evaluation metrics - we might not need to use log loss
            log_loss_score = log_loss(y_test, y_proba, sample_weight=sample_weight_test)
            log_losses.append(log_loss_score)
            accuracies.append(accuracy_score(y_test, model.predict(X_test_scaled)))
            auc_score = roc_auc_score(y_test, y_proba[:, 1], sample_weight=sample_weight_test)
            auc_scores.append(auc_score)
            roc_data[name]['fpr'].append(fpr)
            roc_data[name]['tpr'].append(tpr)
            roc_data[name]['auc'].append(roc_auc)

            # Update the aggregated confusion matrix
            cm = confusion_matrix(y_test, model.predict(X_test_scaled), labels=[0, 1])
            aggregated_cm += cm

        # Calculate average metrics
        avg_log_loss = np.mean(log_losses)
        avg_accuracy = np.mean(accuracies)
        avg_auc = np.mean(auc_scores)

        # Aggregate feature importances and store in DataFrame
        if all_importances:
            avg_importance = np.mean(all_importances, axis=0)
            feature_importance_df = pd.DataFrame({
                'Feature': X.columns,
                f'{name}_Importance': avg_importance
            })
            feature_importance_dfs[name] = feature_importance_df

        # Store the final results
        model_scores[name] = {
            'Average Accuracy': avg_accuracy,
            'Average Log Loss': avg_log_loss,
            'Average AUC': avg_auc,
            'Aggregated Confusion Matrix': aggregated_cm
        }
        y_proba_cv[name] = y_proba_model
        
        
        

    return model_scores, feature_importance_dfs, y_proba_cv, split_indices, roc_data


In [ ]:
#Storing the results from CV

scores, feature_importances, y_proba_cvv,split_indices, roc_data = time_series_validation_binary(
    model_classes=binary_models, 
    X=X, 
    y=y_new,
    n_splits= 5,
    weights = weight_train
)

In [ ]:
scores

In [ ]:
#Storing roc data for ROC curve graph
rf_rocdata = roc_data['RandomForest']
lr_rocdata = roc_data['LogisticRegression']
gb_rocdata = roc_data['GradientBoosting']
xgb_rocdata = roc_data['XGB']

In [ ]:
#Displaying from Random Forest tree

Image(filename='RandomForest_tree.png')

In [ ]:
#Plotting our cross validation splits.

sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(14, 7))


train_color = '#1f77b4'  
test_color = '#ff7f0e'   

for i, (train_indices, test_indices) in enumerate(split_indices):
    ax.barh(y=i, width=max(train_indices)-min(train_indices), left=min(train_indices), 
            color=train_color, height=0.4, label='Train' if i == 0 else "")
    ax.barh(y=i, width=max(test_indices)-min(test_indices), left=min(test_indices), 
            color=test_color, height=0.4, label='Test' if i == 0 else "")

ax.legend(fontsize='large')
ax.set_yticks(range(len(split_indices)))
ax.set_yticklabels([f'CV {i+1}' for i in range(len(split_indices))])
ax.set_xlabel('Sample Index', fontsize='large', fontweight='bold')
ax.set_ylabel('CV Iteration', fontsize='large', fontweight='bold')
ax.set_title('Time Series Split', fontsize='x-large', fontweight='bold')

ax.invert_yaxis()
plt.tight_layout()

plt.show()


In [ ]:
#Storing results and feature importance from the models
random_forest_scores = scores['RandomForest']
random_forest_feature_importances = feature_importances['RandomForest']
gradient_boosting_scores = scores['GradientBoosting']
gradient_boosting_feature_importances = feature_importances['GradientBoosting']


In [ ]:
#Checking top 10 feature importances from each of our models

df_rf_top10 = feature_importances['RandomForest'].nlargest(10, 'RandomForest_Importance')
df_gb_top10 = feature_importances['GradientBoosting'].nlargest(10, 'GradientBoosting_Importance')
df_xgb_top10 = feature_importances['XGB'].nlargest(10, 'XGB_Importance')

df_rf_top10.reset_index(drop=True, inplace=True)
df_gb_top10.reset_index(drop=True, inplace=True)
df_xgb_top10.reset_index(drop=True, inplace=True)
df_top_features = pd.concat([df_rf_top10, df_gb_top10, df_xgb_top10], axis=1)

df_top_features


In [ ]:
#Plotting the confusion matrices

cm_matrix_rf = random_forest_scores['Aggregated Confusion Matrix']

sns.heatmap(cm_matrix_rf, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Random Forest')
plt.show()

In [ ]:
#Confusion matrix GB

cm_matrix_rf = gradient_boosting_scores['Aggregated Confusion Matrix']

sns.heatmap(cm_matrix_rf, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Gradient Boosting')
plt.show()

In [ ]:
#Here we have our hyperparameter optimization parameters to test. We will be using optuna to do that. 



def xgb_param_sample(trial):
    params = {
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.15, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 40),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'gamma': trial.suggest_float("gamma", 0, 2.0),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 100.0, log=True),
        'reg_alpha': trial.suggest_float("reg_alpha", 1e-8, 100.0, log=True),
        'min_child_weight': trial.suggest_int("min_child_weight", 1, 20),
        'n_estimators': trial.suggest_int("n_estimators", 700, 1200),

        # some fixed hyperparameters
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
    }

    return params

def random_forest_param_sample(trial):
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 400, 500),
        'max_depth': trial.suggest_int("max_depth", 1, 100),
        'min_samples_split': trial.suggest_float("min_samples_split", 0.0000001, 0.5, log=True),
        'min_samples_leaf': trial.suggest_float("min_samples_leaf", 0.0000001, 0.5, log=True),
        'max_features': trial.suggest_float('max_features', 0.0, 1.0),
        'min_impurity_decrease': trial.suggest_float("min_impurity_decrease", 0.0000001, 1, log=True),
        'ccp_alpha': trial.suggest_float("ccp_alpha", 0.0000001, 2, log=True),
        'criterion': trial.suggest_categorical("criterion", ['gini', 'entropy', 'log_loss']),
        'n_jobs':trial.suggest_categorical('n_jobs',[-1]),
        'oob_score':trial.suggest_categorical('oob_score',[False]),
        'class_weight': trial.suggest_categorical('class_weight',[None])
 
    }
    
    return params

def gradient_boosting_param_sample(trial):
    params = {
        'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.15, log=True),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 40),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }

    return params

In [ ]:
# Random Forest objective function and study

def random_forest_objective_auc(trial):
    params = random_forest_param_sample(trial)
    tscv = TimeSeriesSplit(n_splits=5)
    auc_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weighttrain = weight_train.iloc[train_index]
        weighttest = weight_train.iloc[test_index]

        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train, sample_weight=weighttrain)

        preds = model.predict_proba(X_test)[:, 1]


        score = roc_auc_score(y_test, preds, sample_weight=weighttest)
        if not np.isnan(score):
            auc_scores.append(score)
    
    if not auc_scores:
        return np.nan

    return np.mean(auc_scores)


rf_auc_study = optuna.create_study(direction='maximize')
rf_auc_study.optimize(random_forest_objective_auc, n_trials=20)


In [ ]:
# XGB objective function and study

def xgb_objective_auc(trial):
    params = xgb_param_sample(trial)
    tscv = TimeSeriesSplit(n_splits=5)
    auc_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weighttrain = weight_train.iloc[train_index]
        weighttest = weight_train.iloc[test_index]

        model = XGBClassifier(**params)
        model.fit(X_train, y_train, sample_weight=weighttrain)

        preds = model.predict_proba(X_test)[:, 1]


        score = roc_auc_score(y_test, preds, sample_weight=weighttest)
        if not np.isnan(score):
            auc_scores.append(score)
    
    if not auc_scores:
        return np.nan

    return np.mean(auc_scores)


xgb_auc_study = optuna.create_study(direction='maximize')
xgb_auc_study.optimize(xgb_objective_auc, n_trials=20)


In [ ]:
# Gradient Boosting objective function and study

def gradient_boosting_objective_auc(trial):
    params = gradient_boosting_param_sample(trial)
    tscv = TimeSeriesSplit(n_splits=5)
    auc_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        weighttrain = weight_train.iloc[train_index]
        weighttest = weight_train.iloc[test_index]

        model = GradientBoostingClassifier(**params)
        model.fit(X_train, y_train, sample_weight=weighttrain)

        preds = model.predict_proba(X_test)[:, 1]


        score = roc_auc_score(y_test, preds, sample_weight=weighttest)
        if not np.isnan(score):
            auc_scores.append(score)
    
    if not auc_scores:
        return np.nan

    return np.mean(auc_scores)


gb_auc_study = optuna.create_study(direction='maximize')
gb_auc_study.optimize(gradient_boosting_objective_auc, n_trials=20)


In [ ]:
#Storing best scores and best params

rf_best_score = rf_auc_study.best_value
# xgb_best_score_auc = xgb_auc_study.best_value
gb_best_score_auc = gb_auc_study.best_value

rf_best_params_auc = rf_auc_study.best_params
# xgb_best_params_auc = xgb_auc_study.best_params
gb_best_params_auc = gb_auc_study.best_params

# Creating dictionary with each model's best parameters
model_configs = {'RandomForest': rf_best_params_auc}
# model_configs['XGB'] = xgb_best_params_auc
model_configs['GradientBoosting'] = gb_best_params_auc

In [ ]:
#Parameter importance Random Forest
optuna.visualization.plot_param_importances(rf_auc_study)

In [ ]:
#Parameter importance Gradient Boosting
optuna.visualization.plot_param_importances(gb_auc_study)

In [ ]:
#RF optm history

optuna.visualization.plot_optimization_history(rf_auc_study)

In [ ]:
#GB optm history

optuna.visualization.plot_optimization_history(gb_auc_study)

In [ ]:
#Displaying RF importance in sample

sns.set_theme(style="whitegrid")
plt.figure(figsize=(8, 6))
top_n = 20
top_features_df = random_forest_feature_importances.sort_values(by='RandomForest_Importance', ascending=False).head(top_n)
sns.barplot(x='RandomForest_Importance', y='Feature', data=top_features_df)

plt.xticks(rotation=45)

plt.tight_layout() 
plt.show()


In [ ]:
#Displaying GB importance 

sns.set_theme(style="whitegrid")
plt.figure(figsize=(8, 6))
top_n = 20
top_features_df_gb = gradient_boosting_feature_importances.sort_values(by='GradientBoosting_Importance', ascending=False).head(top_n)
sns.barplot(x='GradientBoosting_Importance', y='Feature', data=top_features_df_gb)

plt.xticks(rotation=45)

plt.tight_layout() 
plt.show()

In [ ]:
#This is our out of sample evaluation. It will be performed in the test set we held out.

def out_of_sample_evaluation(model_classes, X_train, y_train, X_test, y_test, random_state=1, model_configs=None, use_sample_weights=False, weight_train=None, weight_test=None):

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    results = {}
    y_proba_out_of_sample = {}
    trained_models = {}
    feature_importancess = {}

    for name, model_class in model_classes.items():
        config = model_configs.get(name, {}) if model_configs is not None else {}
        config['random_state'] = random_state

        model = model_class(**config)
        if use_sample_weights:
            model.fit(X_train_scaled, y_train, sample_weight=weight_train)
        else:
            model.fit(X_train_scaled, y_train)

        if name == 'RandomForest':
            feature_names = X_train.columns if isinstance(X_train, pd.DataFrame) else [f'feature_{i}' for i in range(X_train.shape[1])]
            estimator = model.estimators_[0]
            
            export_graphviz(estimator, out_file=f'{name}_tree.dot', 
                            feature_names=feature_names,
                            rounded=True, proportion=False, 
                            precision=2, filled=True)
            os.system(f'dot -Tpng {name}_tree.dot -o {name}_tree.png')

        trained_models[name] = model
        
        if name == 'LogisticRegression':
            if isinstance(X_train, pd.DataFrame):
                feature_names = X_train.columns
            else:
                feature_names = [f'feature_{i}' for i in range(X_train.shape[1])]
            
            coefficients = model.coef_[0]
        
            feature_importance = dict(zip(feature_names, coefficients))
            feature_importancess[name] = feature_importance


        y_proba = model.predict_proba(X_test_scaled)[:, 1]
        y_proba_out_of_sample[name] = y_proba.tolist()

        y_pred_custom_threshold = (y_proba > 0.5).astype(int)

        test_accuracy = accuracy_score(y_test, y_pred_custom_threshold)
        test_auc = roc_auc_score(y_test, y_proba)
        test_log_loss = log_loss(y_test, y_proba, sample_weight=weight_test)
        test_cm = confusion_matrix(y_test, y_pred_custom_threshold)

        results[name] = {
            'Average Accuracy': test_accuracy,
            'Average Log Loss': test_log_loss,
            'Average AUC': test_auc,
            'Aggregated Confusion Matrix': test_cm.tolist(), 
        }

    return results, y_proba_out_of_sample, trained_models, feature_importancess


In [ ]:
#Results with 50% probability cutoff

results, y_proba_out_of_sample,trained_models, feature_importances_out_of_sample = out_of_sample_evaluation(
    model_classes=binary_models,
    X_train=X,
    y_train=y_new,
    X_test=X_test,
    y_test=y_new_test,
    use_sample_weights=True,
    weight_train=weight_train,
    weight_test=weight_test,
    model_configs=model_configs
)
results


In [ ]:
#Results 40% probability cutoff
results_40, y_proba_out_of_sample_40,trained_models_40, feature_importances_out_of_sample_40 = out_of_sample_evaluation(
    model_classes=binary_models,
    X_train=X,
    y_train=y_new,
    X_test=X_test,
    y_test=y_new_test,
    use_sample_weights=True,
    weight_train=weight_train,
    weight_test=weight_test,
    model_configs=model_configs
)
results_40


In [ ]:
cm_matrix_rf_out_of_sample = results['GradientBoosting']['Aggregated Confusion Matrix']

sns.heatmap(cm_matrix_rf_out_of_sample, annot=True, fmt='d', cmap='Blues')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Gradient Boosting: 50% Probability Cutoff')
plt.show()

In [ ]:
cm_matrix_rf_out_of_sample_40 = results_40['GradientBoosting']['Aggregated Confusion Matrix']

sns.heatmap(cm_matrix_rf_out_of_sample, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Random Forest: 40% Probability Cutoff')
plt.show()

In [ ]:
cm_matrix_rf_out_of_sample_40 = results_40['RandomForest']['Aggregated Confusion Matrix']

sns.heatmap(cm_matrix_rf_out_of_sample_40, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Random Forest: 40% Probability Cutoff')
plt.show()

In [ ]:
#Plotting ROC Curve

fpr_rf, tpr_rf, _ = roc_curve(y_new_test, y_proba_out_of_sample['RandomForest'], sample_weight=weight_test)
roc_auc_rf = auc(fpr_rf, tpr_rf)

fpr_lr, tpr_lr, _ = roc_curve(y_new_test, y_proba_out_of_sample['LogisticRegression'], sample_weight=weight_test)
roc_auc_lr = auc(fpr_lr, tpr_lr)

fpr_gb, tpr_gb, _ = roc_curve(y_new_test, y_proba_out_of_sample['GradientBoosting'], sample_weight=weight_test)
roc_auc_gb = auc(fpr_gb, tpr_gb)

plt.figure(figsize=(8, 6))

plt.plot(fpr_rf, tpr_rf, color='blue', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.2f})')
plt.plot(fpr_lr, tpr_lr, color='orange', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.2f})')
plt.plot(fpr_gb, tpr_gb, color='green', lw=2, label=f'Gradient Boosting (AUC = {roc_auc_gb:.2f})')

plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--', label='Random Guess')

plt.title('ROC Curve Comparison', fontsize=16)
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.legend(loc='lower right', fontsize=12, frameon=True, shadow=True)
plt.grid(True, linestyle='--', lw=0.5, alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
#Most important features Logistic regression

lr_most_important = feature_importances_out_of_sample['LogisticRegression']
feature_importances_lr = {feature: abs(coef) for feature, coef in lr_most_important.items()}

sorted_features = sorted(feature_importances_lr.items(), key=lambda item: item[1], reverse=True)

top_20_features = sorted_features[:20]

top_features_df = pd.DataFrame(top_20_features, columns=['Feature', 'Importance'])

top_features_df.sort_values(by='Importance', ascending=False, inplace=True)

plt.figure(figsize=(8, 6))
sns.barplot(x='Importance', y='Feature', data=top_features_df, palette='viridis')

plt.xlabel('Logistic Regression Coefficient Magnitude')
plt.ylabel('Feature')
plt.title('Top 20 Features for Logistic Regression Model')

plt.tight_layout()
plt.show()
